### Import Required Libraries and the Dataset


In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

# Load the IMDb Reviews dataset
dataset, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\Gurdeep\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteIHUTIA\imdb_reviews-tra…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\Gurdeep\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteIHUTIA\imdb_reviews-tes…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\Gurdeep\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteIHUTIA\imdb_reviews-uns…

Dataset imdb_reviews downloaded and prepared to C:\Users\Gurdeep\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


### Function to print random reviews


In [ ]:
def print_random_reviews(dataset, num_reviews=5):
    random_indices = np.random.randint(0, len(dataset), size=num_reviews)
    for index in random_indices:
        review, label = next(iter(dataset.skip(index).take(1)))
        review = review.numpy().decode('utf-8')
        label = "Positive" if label else "Negative"
        print(f"Review: {review}\nSentiment: {label}\n")

# Print random reviews
print("\nRandom Reviews:\n")
print_random_reviews(test_dataset)


### Preprocess the dataset


In [ ]:
def preprocess(text, label):
    text = tf.strings.lower(text)
    text = tf.strings.regex_replace(text, '<br />', ' ')
    text = tf.strings.regex_replace(text, '[^a-zA-Z0-9]', ' ')
    return text, label

# Apply preprocessing to the datasets
train_dataset = train_dataset.map(preprocess)
test_dataset = test_dataset.map(preprocess)


### Print random reviews post preprocessing


In [ ]:
print("\nRandom Reviews:\n")
print_random_reviews(test_dataset)

In [12]:
vocab_size = 10000
encoder = tf.keras.layers.TextVectorization(max_tokens=vocab_size)

# Use a smaller subset for faster processing
train_subset = train_dataset.batch(100).take(100)
encoder.adapt(train_subset.map(lambda text, label: text))

### Build the model


In [ ]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        mask_zero=True),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

### Compile the model


In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

### Train the model


In [ ]:
history = model.fit(train_dataset.shuffle(10000).batch(32),
                    epochs=10,
                    validation_data=test_dataset.batch(32),
                    verbose=1)

### Plot loss and validation accuracy curves


In [ ]:

import matplotlib.pyplot as plt

# Get the loss and validation accuracy values from the history object
loss = history.history['loss']
val_accuracy = history.history['val_accuracy']

# Get the number of epochs
epochs = range(1, len(loss) + 1)

# Create the plot
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss and accuracy')
plt.legend()

# Show the plot
plt.show()


### Evaluate the model


In [ ]:
test_loss, test_accuracy = model.evaluate(test_dataset.batch(32), verbose=0)
print(f'Test Accuracy: {test_accuracy}')

### Please enter a review, then use the above classifier to predict the label.


In [ ]:

review = input("Enter a review: ")
review = preprocess(review, 0)[0]
review = tf.expand_dims(review, axis=0)
prediction = model.predict([review])
sentiment = "Positive" if prediction[0] > 0.5 else "Negative"
print(f"Sentiment: {sentiment}")
